In [25]:

from importlib import import_module
from pathlib import Path
import sys
sys.path.insert(0, "../")
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt

# reload modules
import importlib
import models.fc
import models.cnn
importlib.reload(models.fc)
importlib.reload(models.cnn)

import data
importlib.reload(data)

from data import read_data, read_labels, normalize_img
from models.fc import build_fc_model
from models.cnn import build_cnn_model


In [12]:
# Read MNIST database
(train_data, train_labels), (test_data, test_labels) = tf.keras.datasets.mnist.load_data()

In [13]:
# One hot
train_labels = tf.keras.utils.to_categorical(train_labels, num_classes=10)
test_labels = tf.keras.utils.to_categorical(test_labels, num_classes=10)

In [ ]:
# Peprocessing (Normalization)
print('Raw data pixel value range:', train_data.min(), 'to', train_data.max())
train_data, train_labels = normalize_img(train_data, train_labels)
test_data, test_labels = normalize_img(test_data, test_labels)

print('Normalized datatye: ', type(train_data))
print('Normalized data pixel value range:', train_data.numpy().min(), 'to', train_data.numpy().max())

In [15]:
# Define model
model_type = 'cnn'  # Cambia a 'cnn' para usar la red convolucional

if model_type == 'fc':
    model = build_fc_model()
    # x_train_input, x_test_input = x_train_fc, x_test_fc
elif model_type == 'cnn':
    model = build_cnn_model()
    # x_train_input, x_test_input = x_train_cnn, x_test_cnn
else:
    print(f'{model_type} not supported')

In [ ]:
# Training parameters
OPTIMIZER = 'adam'
LOSS_FUNCTION = 'categorical_crossentropy'
METRICS = ['accuracy']
EPOCHS = 5

model.compile(optimizer=OPTIMIZER,
              loss=LOSS_FUNCTION,
              metrics=METRICS)
model.summary()

In [ ]:
# Train model
history = model.fit(train_data, train_labels, epochs=EPOCHS, validation_data=(test_data, test_labels))

In [ ]:
# Plot the evolution of loss and accuracy
plt.figure(figsize=(12, 5))

# --- Loss plot ---
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.title('Loss Function Evolution')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

# --- Accuracy plot ---
plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.title('Accuracy Evolution')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate model
test_loss, test_acc = model.evaluate(test_data, test_labels, verbose=2)
print(f"Test accuracy: {test_acc:.4f}")

In [ ]:
# Get predictions
start_time = time.time()
predictions = model.predict(test_data)
elapsed_time = time.time() - start_time
print(f"Time per inferecne: {elapsed_time/len(test_data)*1000:.4f} ms")

In [ ]:
# Display prediction i and label
index = 1000
plt.imshow(test_data[index].numpy().squeeze(), cmap='gray')
plt.title(f"Predicted: {np.argmax(predictions[index])}, Label: {np.argmax(test_labels[index])}")
plt.axis('off')
plt.tight_layout()
plt.show()
# plt.savefig('pred_1000.png')

In [ ]:
# Save model
OUTPUT_PATH = Path(f'./../../../saved_model/mnist_{model_type}')
model.save(OUTPUT_PATH / 'SavedModel')
model.save( OUTPUT_PATH / 'model.h5', save_format='h5')